In [ ]:
from osgeo import gdal
import os

In [ ]:
## List input raster files
os.chdir('/Users/flurin/Desktop/MVA-WVtiles-hdf')
rasterFiles = os.listdir(os.getcwd())
#print(rasterFiles)

In [ ]:
#in case you get an error that ds file cannot be convertet get to the directory using cd and from there use the command beneath
#find . -name '.DS_Store' -type f -delete

In [ ]:
for i in range(len(rasterFiles)):
    #Get File Name Prefix
    rasterFilePre = rasterFiles[i][:-3]
    print(rasterFilePre)


    fileExtension = "_BBOX.tif"

    ## Open HDF file
    hdflayer = gdal.Open(rasterFiles[i], gdal.GA_ReadOnly)
    #print(hdflayer.GetSubDatasets())

    # Open raster layer
    #hdflayer.GetSubDatasets()[0][0] - for first layer
    #hdflayer.GetSubDatasets()[1][0] - for second layer ...etc
    subhdflayer = hdflayer.GetSubDatasets()[4][0]
    #print(subhdflayer)

    rlayer = gdal.Open(subhdflayer, gdal.GA_ReadOnly)
    #outputName = rlayer.GetMetadata_Dict()['long_name']

    #Subset the Long Name
    outputName = subhdflayer[92:]

    outputNameNoSpace = outputName.strip().replace(" ","_").replace("/","_")
    outputNameFinal = outputNameNoSpace + rasterFilePre + fileExtension

    #print(outputNameFinal)

    outputFolder = "/Users/flurin/Desktop/MVA-VWtiles-tif/"

    outputRaster = outputFolder + outputNameFinal
    #print("******This is the final outputname: ", outputRaster)

    #collect bounding box coordinates
    HorizontalTileNumber = int(rlayer.GetMetadata_Dict()["HorizontalTileNumber"])
    VerticalTileNumber = int(rlayer.GetMetadata_Dict()["VerticalTileNumber"])

    WestBoundCoord = (10*HorizontalTileNumber) - 180
    NorthBoundCoord = 90-(10*VerticalTileNumber)
    EastBoundCoord = WestBoundCoord + 10
    SouthBoundCoord = NorthBoundCoord - 10

    EPSG = "-a_srs EPSG:4326" #WGS84

    translateOptionText = EPSG+" -a_ullr " + str(WestBoundCoord) + " " + str(NorthBoundCoord) + " " + str(EastBoundCoord) + " " + str(SouthBoundCoord)

    translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
    gdal.Translate(outputRaster,rlayer, options=translateoptions)

    #Display image in QGIS (run it within QGIS python Console) - remove comment to display
    #iface.addRasterLayer(outputRaster, outputNameFinal)